In [1]:
import pickle
import os

from glob import glob
from gensim import models
from nltk.corpus import stopwords
from FlaskApp.transform import *
from googletrans import Translator

import pandas as pd
import numpy as np

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
filepaths = glob('data/scraped/job_*')
filenames = [filepath.split('\\')[-1].split('.')[0] for filepath in filepaths]

In [3]:
jobs = {}
for filepath in filepaths:
    filename = filepath.split('\\')[-1].split('.')[0]
    with open(filepath, 'rb') as file:
        loaded = pickle.load(file)
        for job in loaded:
            if 'Type' not in job.keys():
                job['Type'] = None
            if type(job['Description']) != str:
                print(type(job['Description']))
        jobs[filename] = loaded

In [4]:
import re

In [70]:
df_job = pd.DataFrame({
    'Title': [job['Title'] for key in jobs for job in jobs[key]],
    'Company': [job['Company'] for key in jobs for job in jobs[key]],
    'Area': [job['Area'] for key in jobs for job in jobs[key]],
    'Type': [job['Type'] for key in jobs for job in jobs[key]],
    'Skills': [job['Skills'] for key in jobs for job in jobs[key]],
    'Description': [job['Description'] for key in jobs for job in jobs[key]],
})

df_job.Description = df_job.Description.apply(lambda x: x.lower().split('tentang pekerjaan ini')[-1].split('pronunciation')[0])
df_job.Description = df_job.Description.apply(lambda x: re.sub('[^a-zA-Z0-9&#.+-]', ' ', x))

df_job.Skills = df_job.Skills.apply(lambda x: ' '.join(x) if x != None else '').map(str.lower).apply(lambda x: re.sub('[^a-zA-Z0-9&#.+-]', ' ', x))

In [71]:
df_job.isna().sum()

Title            0
Company          0
Area             0
Type           681
Skills           0
Description      0
dtype: int64

In [72]:
df_job.fillna('', inplace=True)

In [73]:
df_job['Text'] = df_job.Skills + ' ' + df_job.Description

In [74]:
id_stopwords = stopwords.words('indonesian')
en_stopwords = stopwords.words('english')
nltk_stopwords = id_stopwords + en_stopwords

In [75]:
df_job.Text = df_job.Text.apply(lambda x: ' '.join([i for i in x.split() if i not in nltk_stopwords]))
df_job.Text = df_job.Text.apply(lambda x: re.sub(r'\s\W\s', ' ', x))
df_job.Text = df_job.Text.apply(lambda x: re.sub(r'\.\s', ' ', x))

In [76]:
df_job.tail()

,Title,Company,Area,Type,Skills,Description,Text
8367,Senior Full-Stack Engineer,Turing,Indonesia,Jarak Jauh,asp.net asp.net mvc back-end web development ef mern stack model-view-controller mvc oauth sql stored procedures web applications,tentang kami lakeba group adalah salah satu perusahaan teknologi baru yang paling inovatif dan menarik di australia diakui secara internasional oleh the financial times sebagai perusahaan pertumbuhan tinggi asia pasifik selama dua tahun berturut-turut. tim teknik global kami mengembangkan solusi al yang inovatif dan inovatif untuk beberapa layanan keuangan paling menarik dan bergengsi di dunia serta klien perusahaan terkemuka. membangun pertumbuhan dua digit selama bertahun-tahun kami bersemangat untuk mengembangkan tim penjualan kami untuk mempercepat pertumbuhan baik di dalam negeri maupun di as asia tenggara india dan eropa. dalam beberapa tahun terakhir lakeba telah memperluas operasinya secara global dan telah mendirikan kantor di beberapa negara termasuk australia amerika serikat dan indonesia. keputusan untuk membuka kantor di indonesia didorong oleh beberapa faktor termasuk kumpulan besar pekerja berbakat dan terampil lingkungan bisnis yang menguntungkan dan lokasi yang strategis di asia tenggara. kami percaya pada kekuatan pengetahuan kolektif yang terdiri dari tempat kerja inklusif kami investasi kami inovasi kami dan filosofi pemikiran desain kami. memimpin dengan tujuan lakeba memberikan hasil transformatif yang merekayasa teknologi masa depan mempercepat bisnis untuk hari esok yang lebih baik. tentang peran kami mencari pengembang full stack mern yang terampil yang akan bertanggung jawab untuk mengembangkan dan atau merancang produk berbasis saas untuk usaha kami. anda akan bekerja sama dengan pengembang lain dalam membuat memelihara dan memperbarui produk kami. kandidat ideal kami adalah seseorang yang bersedia tumbuh bersama kami. agar anda berhasil dalam peran ini anda harus mahir dalam pengembangan back-end dengan inti .net .net pengetahuan dan pengalaman yang kuat dalam pemrograman aplikasi web menggunakan sql. deskripsi pekerjaan ini adalah kesempatan yang menarik bagi pengembang backend untuk bergabung dengan perusahaan global yang menyenangkan dan berkembang pesat. posisi tersebut akan melapor kepada pemimpin tim pengembangan dan akan bekerja sama erat dengan arsitek solusi. tanggung jawab utama mengembangkan atau memelihara layanan web atau aplikasi web menggunakan tumpukan teknis microsoft termasuk c# asp.net mvc prinsip rest sql server berkolaborasi dengan tim devops dalam mengidentifikasi memperbaiki masalah produksi merancang dan mengimplementasikan integrasi dengan platform pihak ketiga berikut praktik terbaik. melakukan tinjauan kode berkolaborasi dengan arsitek solusi dalam membuat poc proposal solusi dan membantu dengan arsitektur aplikasi berhubungan dengan tim pengembangan front end atau seluler pelaporan berkelanjutan dan pembaruan proyek ke semua pemangku kepentingan yang diperlukan keterampilan yang dibutuhkan mengembangkan dan hosting layanan web kesombongan dan integrasi dengan aplikasi web azure. pengalaman yang kuat dalam berbagai jenis otentikasi otentikasi oauth otentikasi berbasis token openid otentikasi berbasis klaim dan azure active directory c# asp.net layanan web server sql kueri tingkat lanjut penyesuaian kinerja prosedur tersimpan arsitektur mvc kerangka entitas. pengetahuan suara dalam push notifications signalr pengalaman kerja dalam ci cd dan build automation. harus memiliki pengetahuan ef baik untuk memahami teknologi web seperti ajax angular js jquery wcf layanan lainnya dll. pengetahuan inti .net menguntungkan keistimewaan di lakeba bekerja dengan tim berorientasi kinerja yang didorong oleh kepemilikan dan terbuka untuk eksperimen dan teknologi baru produk yang akan anda kerjakan digunakan oleh bank besar di seluruh dunia belajar merancang sistem untuk akurasi efisiensi dan skalabilitas tinggi fokus pada hasil kerja yang berkualitas. buda

In [77]:
dataset = df_job.Text.map(str.split).values

data = []
for word in dataset:
    data.append(word)

def tagged_document(list_of_listwords, tags=None):
    if tags is not None:
        for x, listwords in zip(tags, list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])
    else:
        for x, listwords in enumerate(list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])

data_train = list(tagged_document(data, df_job.Title.values))


In [79]:
d2v_model = models.doc2vec.Doc2Vec(vector_size=120, min_count=10, epochs=60, window=4)
d2v_model.build_vocab(data_train)

2023-05-04 17:27:43,604 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d120,n5,w4,mc10,s0.001,t3>', 'datetime': '2023-05-04T17:27:43.604151', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-05-04 17:27:43,606 : INFO : collecting all words and their counts
2023-05-04 17:27:43,608 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-05-04 17:27:43,868 : INFO : collected 26482 word types and 4557 unique tags from a corpus of 8372 examples and 1464077 words
2023-05-04 17:27:43,869 : INFO : Creating a fresh vocabulary
2023-05-04 17:27:43,914 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 9791 unique words (36.97% of original 26482, drops 16691)', 'datetime': '2023-05-04T17:27:43.914111', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021,

In [80]:
list(d2v_model.wv.index_to_key)

['pengalaman',
 'tim',
 'bisnis',
 'pekerjaan',
 'perusahaan',
 'kerja',
 'memiliki',
 'pelanggan',
 'penjualan',
 'produk',
 'data',
 'keterampilan',
 'manajemen',
 'layanan',
 'komunikasi',
 'design',
 'indonesia',
 'pengembangan',
 'kemampuan',
 'proyek',
 'teknologi',
 'experience',
 'pemasaran',
 'klien',
 'kuat',
 'mengembangkan',
 'development',
 'pengetahuan',
 'proses',
 'desain',
 'terkait',
 'keuangan',
 'solusi',
 'membangun',
 'lowongan',
 'web',
 'persyaratan',
 'sesuai',
 'aplikasi',
 'sistem',
 'bahasa',
 'membantu',
 'mengelola',
 'pasar',
 'tanggung',
 'tujuan',
 'bidang',
 'industri',
 'team',
 'mendukung',
 'peluang',
 'bertanggung',
 'orang',
 'description',
 'platform',
 'digital',
 'management',
 'hubungan',
 'informasi',
 'meningkatkan',
 'utama',
 'software',
 'karyawan',
 'peran',
 'strategi',
 'kebutuhan',
 'work',
 'dunia',
 'job',
 'lingkungan',
 'media',
 'teknis',
 'internal',
 'perangkat',
 'skills',
 'global',
 'mendorong',
 'company',
 'operasi',
 'tam

In [83]:
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-05-04 17:27:57,195 : WARNING : Effective 'alpha' higher than previous training cycles
2023-05-04 17:27:57,196 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 9791 vocabulary and 120 features, using sg=0 hs=0 sample=0.001 negative=5 window=4 shrink_windows=True', 'datetime': '2023-05-04T17:27:57.196633', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-05-04 17:27:58,007 : INFO : EPOCH 0: training on 1464077 raw words (1370118 effective words) took 0.8s, 1694398 effective words/s
2023-05-04 17:27:58,832 : INFO : EPOCH 1: training on 1464077 raw words (1370509 effective words) took 0.8s, 1666338 effective words/s
2023-05-04 17:27:59,616 : INFO : EPOCH 2: training on 1464077 raw words (1370431 effective words) took 0.8s, 1750457 effective words/s
2023-05-04 17:28:00,451 : INFO : EPOCH 3: training on 1464077 raw words

In [84]:
linkedin_words = d2v_model.wv.index_to_key
linkedin_vectors = d2v_model.wv.vectors

with open('data/linkedin_vectors.tsv', 'w', encoding='utf-8') as f:
    for i, word in enumerate(linkedin_words):
        vector_str = '\t'.join([str(num) for num in linkedin_vectors[i]])
        f.write(f'{vector_str}\n')

with open('data/linkedin_metadata.tsv', 'w', encoding='utf-8') as f:
    for word in linkedin_words:
        f.write(f'{word}\n')